In [1]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

c:\Users\user\.conda\envs\langchain-env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import streamlit as st
from streamlit_chat import message # 설치(파이썬 3.8 이상에서만 작동)
import pandas as pd
from sentence_transformers import SentenceTransformer #sentence_transformer 설치 필요
from sklearn.metrics.pairwise import cosine_similarity
import json

2025-10-01 14:29:05.133 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


# SentenceBERT 모델 로드

In [4]:
model = SentenceTransformer('jhgan/ko-sroberta-multitask')

sentences = ["안녕하세요?", "한국어 문장 임베딩을 위한 버트 모델입니다."]
embeddings = model.encode(sentences)

print(embeddings)

[[-0.37510446 -0.77338415  0.59277105 ...  0.579235    0.3268346
  -0.65089685]
 [-0.09361713 -0.18191506 -0.192308   ... -0.03165798  0.30412498
  -0.26793608]]


# 데이터셋 로드

웰니스 대화 스크립트 데이터셋
좋은 질문이에요. 말씀하신 링크는 예전 주소라서 지금은 열리지 않는 것 같고, 현재는 AI-Hub 공식 페이지에서 “웰니스 대화 스크립트 데이터셋”을 내려받을 수 있습니다.
👉 최신 다운로드 페이지: AI-Hub 웰니스 대화 스크립트 데이터셋

⚠️ 참고해야 할 점
- 회원가입 및 로그인 필요: AI-Hub 계정이 있어야 다운로드 가능합니다.
- 내국인만 신청 가능: 안내에 따르면 대한민국 국적자만 데이터 신청이 허용됩니다.
- 안심존 접근: 일부 의료·상담 데이터는 보안 문제 때문에 “안심존(온라인/오프라인)”에서만 접근할 수 있습니다. 즉, 단순히 CSV 파일을 바로 내려받는 방식이 아니라, 지정된 환경에서만 분석할 수 있는 경우가 있습니다.
- 분할 압축: 다운로드 시 여러 개의 .part 파일로 제공되며, 리눅스/WSL 환경에서 병합해야 정상적으로 압축 해제가 가능합니다.

정리
- wellness_dataset_original.csv는 AI-Hub에서 제공하는 원본 파일이 아니라, 누군가가 AI-Hub 데이터셋을 가공해서 만든 버전일 가능성이 큽니다.
- 원본은 AI-Hub에서 신청 후 내려받아야 하고, 직접 CSV 형태로 제공되지 않을 수도 있습니다.

혹시 제가 원본 AI-Hub 데이터셋을 어떻게 CSV 형태로 변환해서 쓸 수 있는지, 예시 코드까지 정리해드릴까요?


https://aihub.or.kr/opendata/keti-data/recognition-laguage/KETI-02-006

In [6]:
df = pd.read_csv('wellness_dataset_original.csv')

df.head()

,구분,유저,챗봇,Unnamed: 3
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,NaN
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,NaN
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.",NaN
3,감정/감정조절이상,꼭 롤러코스터 타는 것 같아요.,NaN,NaN
4,감정/감정조절이상,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,NaN,NaN


# 전처리

In [7]:
df = df.drop(columns=['Unnamed: 3'])

df.head()

,구분,유저,챗봇
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
3,감정/감정조절이상,꼭 롤러코스터 타는 것 같아요.,NaN
4,감정/감정조절이상,롤러코스터 타는 것처럼 기분이 왔다 갔다 해요.,NaN


In [8]:
df = df[~df['챗봇'].isna()]

df.head()

,구분,유저,챗봇
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요."
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.


In [9]:
df.loc[0, '유저']

'제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.'

In [10]:
model.encode(df.loc[0, '유저'])

array([-4.80606586e-01, -2.94869334e-01,  4.37900126e-01, -6.40137732e-01,
        3.28671858e-02, -3.42647552e-01, -5.47486544e-02,  1.73057485e-02,
       -4.08220768e-01, -5.06034076e-01, -1.68733448e-01, -3.98676872e-01,
       -1.24776758e-01, -9.71538499e-02, -1.65286735e-01,  5.72601380e-03,
        6.13692887e-02, -1.91312000e-01,  2.53917396e-01, -5.85019469e-01,
       -2.84425706e-01, -2.32034966e-01, -3.27080637e-01,  6.72976151e-02,
       -1.63742479e-05, -4.72336441e-01, -3.60021859e-01,  2.91879982e-01,
       -6.63861275e-01, -3.10574859e-01,  5.79524994e-01, -3.11722934e-01,
        1.47699649e-02, -2.12172583e-01,  2.22058207e-01, -1.73828915e-01,
       -3.78458321e-01, -4.20398176e-01, -2.38218859e-01,  6.38702437e-02,
       -1.15304179e-01, -2.44564041e-01, -5.00228405e-01,  1.68355450e-01,
       -6.58360064e-01, -8.91941726e-01, -6.26957059e-01, -3.21965486e-01,
       -7.05358505e-01,  3.71447325e-01, -5.45803249e-01,  7.76299089e-02,
        1.09864868e-01,  

# 유저 대화내용 인코딩

In [11]:
df['embedding'] = pd.Series([[]] * len(df)) # dummy

df['embedding'] = df['유저'].map(lambda x: list(model.encode(x)))

df.head()

,구분,유저,챗봇,embedding
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.4806066, -0.29486933, 0.43790013, -0.64013..."
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561579, -0.14506227, 0.29490334, -0.67394..."
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.66520035, -0.081268415, 1.0945567, 0.10579..."
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.,"[-0.7679059, 0.4652074, 0.52850705, -0.5076047..."
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.,"[-0.20277742, -0.37413925, 0.040532034, -0.862..."


In [12]:
df.to_csv('wellness_dataset.csv', index=False)

# 간단한 챗봇

In [13]:
text = '요즘 머리가 아프고 너무 힘들어'

embedding = model.encode(text)

In [14]:
df['distance'] = df['embedding'].map(lambda x: cosine_similarity([embedding], [x]).squeeze())

df.head()

,구분,유저,챗봇,embedding,distance
0,감정/감정조절이상,제 감정이 이상해진 것 같아요. 남편만 보면 화가 치밀어 오르고 감정 조절이 안되요.,감정이 조절이 안 될 때만큼 힘들 때는 없는 거 같아요.,"[-0.4806066, -0.29486933, 0.43790013, -0.64013...",0.448967
1,감정/감정조절이상,더 이상 내 감정을 내가 컨트롤 못 하겠어.,저도 그 기분 이해해요. 많이 힘드시죠?,"[-1.1561579, -0.14506227, 0.29490334, -0.67394...",0.490199
2,감정/감정조절이상,하루종일 오르락내리락 롤러코스터 타는 기분이에요.,"그럴 때는 밥은 잘 먹었는지, 잠은 잘 잤는지 체크해보는 것도 좋아요.","[-0.66520035, -0.081268415, 1.0945567, 0.10579...",0.352131
15,감정/감정조절이상/화,평소 다른 일을 할 때도 비슷해요. 생각한대로 안되면 화가 나고…그런 상황이 지속되...,화가 폭발할 것 같을 때는 그 자리를 피하는 것도 좋은 방법이라고 생각해요.,"[-0.7679059, 0.4652074, 0.52850705, -0.5076047...",0.422284
16,감정/감정조절이상/화,예전보다 화내는 게 과격해진 거 같아.,정말 힘드시겠어요. 화는 남에게도 스스로에게도 상처를 주잖아요.,"[-0.20277742, -0.37413925, 0.040532034, -0.862...",0.315118


In [60]:
answer = df.loc[df['distance'].idxmax()]

print('구분', answer['구분'])
print('유사한 질문', answer['유저'])
print('챗봇 답변', answer['챗봇'])
print('유사도', answer['distance'])

구분 증상/편두통
유사한 질문 요즘은 머리가 한쪽만 지그시 누르는 것처럼 무겁고 아파요.
챗봇 답변 으으, 머리가 아프면 정말 힘들죠. 그 마음 정말 이해해요.
유사도 0.8296288251876831
